In [1]:
#Comandos previos
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

In [2]:
from utils import compute_metrics
from datasets import load_dataset
dataset = load_dataset("imdb")

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-1767aa7e0d445794.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-e14c433f0c95fb3c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-c7b1add269362cb1.arrow


In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-89968d56dfd03f35.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-8a52e5d1975fcd86.arrow


In [5]:
# PEFT: Parametros de ajuste fino eficiente
# task_type: Especifica el tipo de tarea el cual  el modelo se ajustara
# r son las dimensiones de las matrices A y B
# lora_alpha es el factor de escala, determina la relacion de los pesos A y B
# en relacion con los parametros originales del modelo
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
# Insertamos las matrices A y B en el modelo (get_peft_model)
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [7]:
# ENTRENAMIENTO Y EVALUACION DEL MODELO
# evaluate.load calcula e informa metricas, es una fx de precision sencilla
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
# UTILS
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_auc_score

def compute_metrics(pred):
    labels = pred.label_ids
    prediction=pred.predictions
    preds = prediction.argmax(-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    precision = tp / (tp + fp) 
    recall = tp / (tp + fn)
    sn = tp / (tp + fp)       
    sp = tn / (tn + fp)  # true negative rate
    mcc = matthews_corrcoef(labels, preds)
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall)
    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'sn': sn,
        'sp': sp,
        'accuracy': acc,
        'mcc': mcc
    }

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=20,)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

trainer.save_model("./BERT_LoRA")

The following columns in the training set don't have a corresponding argument in `PeftModelForSequenceClassification.forward` and have been ignored: text, token_type_ids. If text, token_type_ids are not expected by `PeftModelForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Auc,Precision,Recall,F1,Sn,Sp,Accuracy,Mcc
1,No log,0.696316,0.493340,0.484425,0.924180,0.635659,0.484425,0.062500,0.483000,-0.026269
2,No log,0.685058,0.539687,0.666667,0.151639,0.247078,0.666667,0.927734,0.549000,0.126302
3,No log,0.679707,0.541016,0.734940,0.125000,0.213660,0.734940,0.957031,0.551000,0.148628
4,0.696100,0.673374,0.586658,0.713542,0.280738,0.402941,0.713542,0.892578,0.594000,0.219951
5,0.696100,0.665103,0.638448,0.713355,0.448770,0.550943,0.713355,0.828125,0.643000,0.300072
6,0.696100,0.652674,0.653272,0.712610,0.497951,0.586248,0.712610,0.808594,0.657000,0.323235
7,0.696100,0.634150,0.653544,0.773234,0.426230,0.549538,0.773234,0.880859,0.659000,0.346158
8,0.660200,0.599129,0.711738,0.706612,0.700820,0.703704,0.706612,0.722656,0.712000,0.423571
9,0.660200,0.563566,0.724545,0.746544,0.663934,0.702820,0.746544,0.785156,0.726000,0.452925
10,0.660200,0.531163,0.739658,0.737500,0.725410,0.731405,0.737500,0.753906,0.740000,0.479562


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForSequenceClassification.forward` and have been ignored: text, token_type_ids. If text, token_type_ids are not expected by `PeftModelForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForSequenceClassification.forward` and have been ignored: text, token_type_ids. If text, token_type_ids are not expected by `PeftModelForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForSequenceClassification.forward` and have been ignored: text, token_type_ids. If text, token_type_ids are not expected by `PeftModelForSequenceClassification.forward